## Importing libraries

In [1]:
from tensorflow.keras.models import Sequential #to initialize models
from tensorflow.keras.layers import Dense #adding layers
from tensorflow.keras.layers import Conv2D #convolution layer
from tensorflow.keras.layers import MaxPool2D #maxpooling
from tensorflow.keras.layers import Flatten 


## Initializing model

In [2]:
model=Sequential()

## Adding convolution layer

In [3]:
 model.add(Conv2D(32,3,3,input_shape=(64,64,3),activation='relu'))
#1st parameter in conv2D = no. of Feature detectors
#2nd &3rd parmater = size of feat. Detect.
#4th parameter = Expected input image shape
#5th parameter =Activation fnctn.
# 64,64,3 for rgb and 64,64,1 for grey scale

## Adding Max pooling layer

In [4]:
model.add(MaxPool2D(pool_size=(2,2)))

## Flatten

In [5]:
model.add(Flatten()) #convert n to 1-dim

## ANN 

In [6]:
model.add(Dense(units=128,activation='relu',kernel_initializer='random_uniform'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 21, 21, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               409728    
Total params: 410,624
Trainable params: 410,624
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.add(Dense(units=3,activation='softmax',kernel_initializer='random_uniform'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 21, 21, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               409728    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 411,011
Trainable params: 411,011
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

## Loading the dataset

In [11]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [12]:
train_datagen= ImageDataGenerator(rescale=1./255,shear_range=0.2,horizontal_flip=True,zoom_range=0.2)

In [13]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [14]:
x_train=train_datagen.flow_from_directory(r'C:\Users\PC\Internship\Weed dataset\TRAIN',target_size=(64,64),batch_size=16,class_mode='categorical')

Found 309 images belonging to 3 classes.


In [15]:
x_test=train_datagen.flow_from_directory(r'C:\Users\PC\Internship\Weed dataset\TEST',target_size=(64,64),batch_size=16,class_mode='categorical')

Found 76 images belonging to 3 classes.


In [16]:
#more than 2 category then class mode= categorical

In [17]:
print(x_train.class_indices)

{'Hedge bindweed train': 0, 'grass train': 1, 'stinging nettle train': 2}


In [18]:
model.fit_generator(x_train,steps_per_epoch=309,epochs=10,validation_data=x_test,validation_steps=76)

Epoch 1/10
309/309 [==============================] - 44s 144ms/step - loss: 0.4002 - accuracy: 0.8375 - val_loss: 0.2892 - val_accuracy: 0.8659
Epoch 2/10
309/309 [==============================] - 50s 163ms/step - loss: 0.1963 - accuracy: 0.9297 - val_loss: 0.2303 - val_accuracy: 0.8841
Epoch 3/10
309/309 [==============================] - 52s 167ms/step - loss: 0.1339 - accuracy: 0.9557 - val_loss: 0.2621 - val_accuracy: 0.8893
Epoch 4/10
309/309 [==============================] - 50s 163ms/step - loss: 0.0865 - accuracy: 0.9669 - val_loss: 0.2487 - val_accuracy: 0.8971
Epoch 5/10
309/309 [==============================] - 51s 165ms/step - loss: 0.0536 - accuracy: 0.9820 - val_loss: 0.4057 - val_accuracy: 0.8659
Epoch 6/10
309/309 [==============================] - 51s 164ms/step - loss: 0.0376 - accuracy: 0.9887 - val_loss: 0.4578 - val_accuracy: 0.8633
Epoch 7/10
309/309 [==============================] - 54s 174ms/step - loss: 0.0340 - accuracy: 0.9899 - val_loss: 0.4246 - val_ac

In [19]:
model.save('weed.h5')

In [2]:
from tensorflow.keras.models import load_model
import numpy as np

import cv2
model=load_model('weed.h5')
from skimage.transform import resize

def detect(frame):
    try:
        img= resize(frame,(64,64))
        #print("dim",img)
        img=np.expand_dims(img,axis=0)
         #print("dim2",img)
        if(np.max(img)>1):
            img= img/255.0
            
        prediction=model.predict(img)
        print(prediction)
        print(model.predict_classes(img))
    except AttributeError:
        print("Shape Not found")
        
frame=cv2.imread(r'C:\Users\PC\Internship\Weed dataset\TEST\grass test\148.jpg')
data=detect(frame)

[[2.1677381e-05 6.8701494e-01 3.1296337e-01]]
[1]


In [3]:
frame=cv2.imread(r'C:\Users\PC\Internship\Weed dataset\TEST\Hedge bindweed test\ii.jpg')
data=detect(frame)

[[9.8653144e-01 1.3454662e-02 1.3840561e-05]]
[0]


In [5]:
frame=cv2.imread(r'C:\Users\PC\Internship\Weed dataset\TEST\stinging nettle test\2.jpg')
data=detect(frame)

[[1.18841834e-01 1.16510205e-06 8.81157041e-01]]
[2]


# Done!